In [ ]:
import pandas as pd
import geopandas as gpd

# Load hydrography data

In [ ]:
hydrography = gpd.read_file("zip://shp_water_dnr_hydrography.zip")

In [ ]:
hydrography.head()
for col in hydrography.columns:
    print(col)

In [ ]:
for i in hydrography.is_valid:
    if i == False:
        print("poly is false")


In [ ]:
# Locate all invalid gometries and drop them from the dataset
hydro_drop_invalid = hydrography.loc[hydrography['geometry'].is_valid, :]

In [ ]:
for i in hydro_drop_invalid.is_valid:
    if i == False:
        print("poly is false")

# Load 7 county metro

In [ ]:
metro = gpd.read_file("zip://shp_bdry_metro_counties_and_ctus.zip")
metro.plot()

In [ ]:
metro.head()

In [ ]:
metro_dissolve = metro.dissolve(by = "CO_NAME")
metro_dissolve.plot()

In [ ]:
metro_dissolve.is_valid

# Clip hydro to metro

In [ ]:
# Varifying the projections
print(metro_dissolve.crs)
print(hydrography.crs)

In [ ]:
# Checking for invalid polygon geometries since the clip operation did not work
for i in hydrography.is_valid:
    if i == False:
        print("poly is false")

In [ ]:
# https://stackoverflow.com/questions/63955752/topologicalerror-the-operation-geosintersection-r-could-not-be-performed
hydro_valid = hydrography.buffer(0)

In [ ]:
# Checking for invalid polygon geometries since the clip operation did not work
for i in hydro_valid.is_valid:
    if i == False:
        print("poly is false")

In [ ]:
hydro_valid.head()

In [ ]:
hydro_clip = gpd.clip(hydro_drop_invalid, metro_dissolve)

In [ ]:
hydro_clip.plot()

In [ ]:
hydro_lake = hydro_clip.loc[hydro_clip["wb_class"] == "Lake or Pond"]

In [ ]:
hydro_lake.head()

In [ ]:
hydro_clip.head()

#hydro_lake = hydro_clip.loc[hydro_clip["wb_class"] == "Lake or Pond"]
#hydro_lake.plot()

# Load 2014, 2016, 2018 water files

In [ ]:
water2018 = gpd.read_file("zip://impaired_2018_lakes.zip")
water2016 = gpd.read_file("zip://impaired_2016_lakes.zip")
water2014 = gpd.read_file("zip://impaired_2014_lakes.zip")

In [ ]:
water2018.plot()

In [ ]:
for col in water2018.columns:
    print(col)

In [ ]:
water2018 = water2018.drop(["CAT", "CAT_DESC", "REACH_DESC", "USE_CLASS", "AREA_ACRES", "AFFECTED_U", "LIKE_MEET", "NON_POLL", 
                            "NAT_BACK", "ADD_MON", "APPROVED", "NEEDS_PLN", "IMP_PARAM", "NEW_IMPAIR", "HUC_8", "HUC_8_NAME", "HUC_4", "BASIN", 
                            "TRIBAL_INT", "INDIAN_RES", "AMMONIA", "CHLORIDE", "FISHESBIO", "HG_F", "HG_W", 
                            "NUTRIENTS", "PCB_F", "PFOS_F", "Shape_Leng", "Shape_Area"], axis = 1)


In [ ]:
water2018

In [ ]:
for col in water2016.columns:
    print(col)

In [ ]:
water2016 = water2016.drop(["CAT", "DATASET_NA", "REACH_DESC", "USE_CLASS", "AREA_ACRES", "AFFECTED_U", "TMDL_NOT_R", "TMDL_NOT_1", "IMPAIR_PAR", "IMPAIR_P_1", "NEW_IMPAIR", "NEW_IMPA_1", 
                "TMDL_APPRO", "TMDL_APP_1", "TMDL_NEEDE", "TMDL_NEE_1", "HUC_8", "HUC_8_NAME", "HUC_4", 
                "BASIN", "TRIBAL_INT", "INDIAN_RES", "CHLORIDE", "FISHESBIO", "HG_F", "HG_W", 
                "NUTRIENTS", "PCB_F", "PFOS_F", "SHAPE_Leng", "SHAPE_Area"], axis = 1)

In [ ]:
water2016

In [ ]:
for col in water2014.columns:
    print(col)

In [ ]:
water2014 = water2014.drop(["LOCATION", "ACRES", "CAT", "AFFECTED_U", "NOPLN", "APPROVED", "NEEDSPLN", "IMPAIR_PAR", "NEW_2014", 
                "HUC8", "HUC8_NAME", "HUC4", "BASIN", "WDWMO_NAME", "WDWMO_TYPE", "Chloride", 
                "HgF", "HgW", "Nutrients", "PCBF", "PFOS_W", "SHAPE_Leng", "Shape_Le_1", "Shape_Area"], axis = 1)
water2014

In [ ]:
water2014 = water2014.rename(columns = {"WATER_NAME" : "NAME", "ALL_COUNTI" : "COUNTY"})
water2014

# Clipping impaired waters layers to 7 county metro

In [ ]:
# Checking for invalid polygon geometries since the clip operation did not work
for i in water2018.is_valid:
    if i == False:
        print("poly is false")

In [ ]:
water2018_drop_invalid = water2018.loc[water2018['geometry'].is_valid, :]

In [ ]:
for i in water2018_drop_invalid.is_valid:
    if i == False:
        print("poly is false")

In [ ]:
water2018_drop_invalid

In [ ]:
for i in water2016.is_valid:
    if i == False:
        print("poly is false")

In [ ]:
water2016_drop_invalid = water2016.loc[water2016['geometry'].is_valid, :]

In [ ]:
for i in water2016_drop_invalid.is_valid:
    if i == False:
        print("poly is false")

In [ ]:
water2016_drop_invalid

In [ ]:
for i in water2014.is_valid:
    if i == False:
        print("poly is false")

In [ ]:
water2014_drop_invalid = water2014.loc[water2014['geometry'].is_valid, :]

In [ ]:
for i in water2014_drop_invalid.is_valid:
    if i == False:
        print("poly is false")

In [ ]:
water2014_drop_invalid

In [ ]:
water2018_clip = gpd.clip(water2018_drop_invalid, metro_dissolve)

In [ ]:
water2018_clip.head()

In [ ]:
water2016_clip = gpd.clip(water2016_drop_invalid, metro_dissolve)

In [ ]:
water2016_clip.head()

In [ ]:
water2014_proj = water2014_drop_invalid.to_crs('EPSG:26915')

In [ ]:
water2014_clip = gpd.clip(water2014_proj, metro_dissolve)

In [ ]:
water2014_clip.head()

# Load water 2020 data csv

In [ ]:
water2020 = gpd.read_file("wq-iw1-65.csv")


In [ ]:
for col in water2020.columns:
    print(col)

In [ ]:
water2020 = water2020[["Water body name", "AUID", "Water body type", "geometry"]]


In [ ]:
water2020_lake = water2020.loc[(water2020["Water body type"] == "Lake")]



In [ ]:
water2020_lake = water2020_lake[["Water body name", "AUID", "geometry"]]

In [ ]:
# Renaming the columns to match the two other datasets
water2020_lake = water2020_lake.rename(columns = {"Water body name" : "NAME"})

water2020_lake

# Trying to remove duplicates

In [ ]:
#join test2 to water2018
# output 1946 rows...there are duplicates...???
jointest = test2.merge(water2018, on = "AUID")

jointest

In [ ]:
test3 = test2.groupby("AUID", as_index = False)["Pollutant or stressor"].apply(";".join)
test3

In [ ]:
#join test3 back to test2

test4 = test2.merge(test3, how = "inner", on = "AUID")
test4

In [ ]:
# Loading the draft 2020 impaired water list
water2020 = gpd.read_file("wq-iw1-65.csv")

# retreiving all the column names
for col in water2020.columns:
    print(col)

# Sampling the data, to get a look at what we are dealing with
water2020.head()

# Selecting out the columns we want to keep
water2020 = water2020[["Water body name", "AUID", "Water body type", "Use Class", "Pollutant or stressor", "geometry"]]
water2020.head()

# Selecting out only the lake features to keep in line with the 3 other shapefiles we are already working with
water2020_lake = water2020.loc[(test["Water body type"] == "Lake")]

# Removing duplicate records --> grouping by AUID, and joining the polluatant or stressor field with ";" when they differ
# for the repeated AUID. This happens because a lake can have more than one stressor to get on the impaired water list.
water2020_remove_duplicates = water2020_lake.groupby("AUID", as_index = False)["Pollutant or stressor"].apply(";".join)
water2020_remove_duplicates

# Failed attempt to join the other needed columns back to the dataset after removing the duplicates
jointest1 = water2020_remove_duplicates.merge(water2020_lake, how = "inner", on = "AUID")
jointest1

# Attempt to get geometry for the 2020 dataset from the 2018 dataset
# Have not figured out the best way to do this yet. 
jointest2 = water2020_remove_duplicates.merge(water2018, on = "AUID")
jointest2



# Load blockgroup data and intersecting it with the water data

In [ ]:
blockgroups_df = gpd.read_file('zip://tl_2019_27_bg.zip')
print(f'Loaded {len(blockgroups_df):,} block groups')

In [ ]:
blockgroups_df.plot()

In [ ]:
print(blockgroups_df.crs)
print(water2018.crs)

In [ ]:
bg_proj = blockgroups_df.to_crs('EPSG:26915')
bg_proj.plot()

In [ ]:
water2018_intersect_bg_proj = gpd.overlay(water2018, bg_proj, how='intersection')
water2018_intersect_bg_proj.plot()
water2018_intersect_bg_proj.head()

In [ ]:
### Code Review Comments ###
# add columns and then remove duplicates again
# find lakes that have been added/removed, and look at the traffic at that spot now
# Smaller scale? Just one city
# function for selecting data with block groups

# Complete Code so far (11/6/2020)

In [ ]:
'''
Author: Nicole Dunn

This python script purpose is to clean spatial data files that will be used in 
future analysis. Cleaning the data includes removing fields that are not needed, 
removing invalid geometries from the geodataframes, adding geometery where there 
is none, and clipping all the data to the 7 county metro as the area of interest.
'''

import pandas as pd
import geopandas as gpd

# Loading all shapefile datasets in as geopandas dataframes
hydrography = gpd.read_file("zip://shp_water_dnr_hydrography.zip")
water2018 = gpd.read_file("zip://impaired_2018_lakes.zip")
water2016 = gpd.read_file("zip://impaired_2016_lakes.zip")
water2014 = gpd.read_file("zip://impaired_2014_lakes.zip")
metro = gpd.read_file("zip://shp_bdry_metro_counties_and_ctus.zip")


# Cleaning the metro dataset, dissolving on the county name. 
metro_dissolve = metro.dissolve(by = "CO_NAME")

# There are many columns in this dataset,
# this is to be able to see all of them to easily
# choose the ones we will keep
columns18 =[]
columns16 =[]
columns14 =[]

for col in water2018.columns:
    columns18.append(col)
    
for col in water2016.columns:
    columns16.append(col)
    
for col in water2014.columns:
    columns14.append(col)

# Dropping all the unnecessary columns
water2018 = water2018.drop(["CAT", "CAT_DESC", "REACH_DESC", "USE_CLASS", "AFFECTED_U", "LIKE_MEET", 
                            "NON_POLL", "NAT_BACK", "ADD_MON", "APPROVED", "NEEDS_PLN", "IMP_PARAM", "NEW_IMPAIR", 
                            "HUC_8", "HUC_8_NAME", "HUC_4", "BASIN", "TRIBAL_INT", "INDIAN_RES", "AMMONIA", "CHLORIDE", 
                            "FISHESBIO", "HG_F", "HG_W", "NUTRIENTS", "PCB_F", "PFOS_F", "Shape_Leng", "Shape_Area"], axis = 1)

water2016 = water2016.drop(["CAT", "DATASET_NA", "REACH_DESC", "USE_CLASS", "AFFECTED_U", "TMDL_NOT_R", 
                            "TMDL_NOT_1", "IMPAIR_PAR", "IMPAIR_P_1", "NEW_IMPAIR", "NEW_IMPA_1", "TMDL_APPRO", "TMDL_APP_1", 
                            "TMDL_NEEDE", "TMDL_NEE_1", "HUC_8", "HUC_8_NAME", "HUC_4", "BASIN", "TRIBAL_INT", "INDIAN_RES", 
                            "CHLORIDE", "FISHESBIO", "HG_F", "HG_W", "NUTRIENTS", "PCB_F", "PFOS_F", "SHAPE_Leng", "SHAPE_Area"], axis = 1)

water2014 = water2014.drop(["LOCATION", "CAT", "AFFECTED_U", "NOPLN", "APPROVED", "NEEDSPLN", "IMPAIR_PAR", 
                            "NEW_2014", "HUC8", "HUC8_NAME", "HUC4", "BASIN", "WDWMO_NAME", "WDWMO_TYPE", "Chloride", 
                            "HgF", "HgW", "Nutrients", "PCBF", "PFOS_W", "SHAPE_Leng", "Shape_Le_1", "Shape_Area"], axis = 1)

# Renaming the columns to match the two other datasets
water2014 = water2014.rename(columns = {"WATER_NAME" : "NAME", "ALL_COUNTI" : "COUNTY", "ACRES" : "AREA_ACRES"})


# Locate all invalid gometries and drop them from the dataset
water2018_drop_invalid = water2018.loc[water2018['geometry'].is_valid, :]

water2016_drop_invalid = water2016.loc[water2016['geometry'].is_valid, :]

water2014_drop_invalid = water2014.loc[water2014['geometry'].is_valid, :]


# Clipping the three impaired water files to the 7 county metro
water2018_clip = gpd.clip(water2018_drop_invalid, metro_dissolve)
water2016_clip = gpd.clip(water2016_drop_invalid, metro_dissolve)

# 2014 needed to be reprojected - then clip was performed
water2014_proj = water2014_drop_invalid.to_crs('EPSG:26915')
water2014_clip = gpd.clip(water2014_proj, metro_dissolve)


# Locate all invalid gometries and drop them from the dataset
hydro_drop_invalid = hydrography.loc[hydrography['geometry'].is_valid, :]

# Clipping hydro to the 7 county metro
hydro_clip = gpd.clip(hydro_drop_invalid, metro_dissolve)

# Narrowing down the number of features in the hydro layer to only lakes and ponds
hydro_lake = hydro_clip.loc[hydro_clip["wb_class"] == "Lake or Pond"]


# Load water 2020 data csv, selecting out the columns that we want and addinga geometry column
# and pulling out only the lake features.
water2020 = gpd.read_file("wq-iw1-65.csv")
water2020 = water2020[["Water body name", "AUID", "County", "Water body type", "geometry"]]
water2020_lake = water2020.loc[(water2020["Water body type"] == "Lake")]

# Dropping the "water body type" field since it is no longer needed
water2020_lake = water2020_lake[["AUID", "Water body name", "County", "geometry"]]

# Renaming the columns to match the two other datasets
water2020_lake = water2020_lake.rename(columns = {"Water body name" : "NAME", "County" : "COUNTY"})

In [ ]:
water2020_lake.head()

In [ ]:
counties = ["Anoka", "Hennepin", "Ramsey", "Washington", "Carver", "Scott", "Dakota"]

In [ ]:
water2020_metro = water2020_lake.loc[(water2020_lake["COUNTY"].isin(counties))]

In [ ]:
water2020_metro["COUNTY"].unique()

In [ ]:
water2020_metro = water2020_lake.loc[(water2020_lake["COUNTY"] == "Anoka")]

In [ ]:
water2020_metro = water2020_metro.append(water2020_lake.loc[(water2020_lake["COUNTY"] == "Hennepin")])

In [ ]:
water2020_metro= water2020_metro.append(water2020_lake.loc[(water2020_lake["COUNTY"] == "Ramsey")])

In [ ]:
water2020_metro = water2020_metro.append(water2020_lake.loc[(water2020_lake["COUNTY"] == "Washington")])

In [ ]:
water2020_metro = water2020_metro.append(water2020_lake.loc[(water2020_lake["COUNTY"] == "Carver")])

In [ ]:
water2020_metro = water2020_metro.append(water2020_lake.loc[(water2020_lake["COUNTY"] == "Scott")])

In [ ]:
water2020_metro = water2020_metro.append(water2020_lake.loc[(water2020_lake["COUNTY"] == "Dakota")])

In [ ]:
water2020_metro

In [ ]:
water2020_clean = water2020_metro.drop_duplicates(subset = ["AUID"])

In [ ]:
water2020_clean

In [ ]:
water2014_clip.head()

In [ ]:
water2016_clip.head()

In [ ]:
water2018_clip.head()

In [ ]:
water2018_clip["AREA_ACRES"].min()

In [ ]:
dfs = [water2014_clip, water2016_clip, water2018_clip]

for df in dfs:
    #print(df)
    #print(type(df))
    smallest_lake = df["AREA_ACRES"].min()
    minimum = 0
    if smallest_lake > minimum:
        minimum = smallest_lake
    else:
        pass

In [ ]:
dfs = [water2014_clip, water2016_clip, water2018_clip]

def find_min(dfs):
    for df in dfs:
        smallest_lake = df["AREA_ACRES"].min()
        minimum = 0
        if smallest_lake > minimum:
            minimum = smallest_lake
        else:
            pass
    return minimum

In [ ]:
find_min(dfs)

In [ ]:
for df in dfs:
    df["status"] = "Impaired"

In [ ]:
water2018_clip.head()

In [ ]:
hydro_lake.head()

In [ ]:
hydro_lake = hydro_lake.loc[(hydro_lake["acres"] >= minimum)]

In [ ]:
for col in hydro_lake.columns:
    print(col)

In [ ]:
hydro_clean = hydro_lake.drop(["fw_id", "dowlknum", "sub_flag", "wb_class", "lake_class", "shore_mi", "center_utm", "center_u_1",
                               "dnr_region", "fsh_office", "outside_mn", "delineated", "delineatio", "delineat_1", "delineat_2", 
                               "approved_b", "approval_d", "approval_n", "has_flag", "flag_type", "publish_da", "lksdb_basi", "has_wld_fl",
                               "wld_flag_t", "created_us", "created_da", "last_edite", "last_edi_1", "ow_use", "pwi_class", "map_displa", 
                               "shape_Leng", "shape_Area", "INSIDE_X", "INSIDE_Y", "in_lakefin"], axis = 1)


In [ ]:
hydro_clean["status"] = ""

In [ ]:
hydro_clean

# Code as of 11/16/2020

In [ ]:
'''
Author: Nicole Dunn

This python script purpose is to clean spatial data files that will be used in 
future analysis. Cleaning the data includes removing fields that are not needed, 
removing invalid geometries from the geodataframes, adding geometery where there 
is none, and clipping all the data to the 7 county metro as the area of interest.
'''

import pandas as pd
import geopandas as gpd

# Loading all shapefile datasets in as geopandas dataframes
hydrography = gpd.read_file("zip://shp_water_dnr_hydrography.zip")
water2018 = gpd.read_file("zip://impaired_2018_lakes.zip")
water2016 = gpd.read_file("zip://impaired_2016_lakes.zip")
water2014 = gpd.read_file("zip://impaired_2014_lakes.zip")
metro = gpd.read_file("zip://shp_bdry_metro_counties_and_ctus.zip")

###
### CLEANING AND CLIPPING IMPAIRED WATER 2014, 2016, AND 2018
###

# Cleaning the metro dataset, dissolving on the county name. 
metro_dissolve = metro.dissolve(by = "CO_NAME")

# Dropping all the unnecessary columns
water2018 = water2018.drop(["CAT", "CAT_DESC", "REACH_DESC", "USE_CLASS", "AFFECTED_U", "LIKE_MEET", 
                            "NON_POLL", "NAT_BACK", "ADD_MON", "APPROVED", "NEEDS_PLN", "IMP_PARAM", "NEW_IMPAIR", 
                            "HUC_8", "HUC_8_NAME", "HUC_4", "BASIN", "TRIBAL_INT", "INDIAN_RES", "AMMONIA", "CHLORIDE", 
                            "FISHESBIO", "HG_F", "HG_W", "NUTRIENTS", "PCB_F", "PFOS_F", "Shape_Leng", "Shape_Area"], axis = 1)

water2016 = water2016.drop(["CAT", "DATASET_NA", "REACH_DESC", "USE_CLASS", "AFFECTED_U", "TMDL_NOT_R", 
                            "TMDL_NOT_1", "IMPAIR_PAR", "IMPAIR_P_1", "NEW_IMPAIR", "NEW_IMPA_1", "TMDL_APPRO", "TMDL_APP_1", 
                            "TMDL_NEEDE", "TMDL_NEE_1", "HUC_8", "HUC_8_NAME", "HUC_4", "BASIN", "TRIBAL_INT", "INDIAN_RES", 
                            "CHLORIDE", "FISHESBIO", "HG_F", "HG_W", "NUTRIENTS", "PCB_F", "PFOS_F", "SHAPE_Leng", "SHAPE_Area"], axis = 1)

water2014 = water2014.drop(["LOCATION", "CAT", "AFFECTED_U", "NOPLN", "APPROVED", "NEEDSPLN", "IMPAIR_PAR", 
                            "NEW_2014", "HUC8", "HUC8_NAME", "HUC4", "BASIN", "WDWMO_NAME", "WDWMO_TYPE", "Chloride", 
                            "HgF", "HgW", "Nutrients", "PCBF", "PFOS_W", "SHAPE_Leng", "Shape_Le_1", "Shape_Area"], axis = 1)

# Renaming the columns to match the two other datasets
water2014 = water2014.rename(columns = {"WATER_NAME" : "NAME", "ALL_COUNTI" : "COUNTY", "ACRES" : "AREA_ACRES"})


# Locate all invalid gometries and drop them from the dataset
water2018_drop_invalid = water2018.loc[water2018['geometry'].is_valid, :]

water2016_drop_invalid = water2016.loc[water2016['geometry'].is_valid, :]

water2014_drop_invalid = water2014.loc[water2014['geometry'].is_valid, :]


# Clipping the three impaired water files to the 7 county metro
water2018_clip = gpd.clip(water2018_drop_invalid, metro_dissolve)
water2016_clip = gpd.clip(water2016_drop_invalid, metro_dissolve)

# 2014 needed to be reprojected - then clip was performed
water2014_proj = water2014_drop_invalid.to_crs('EPSG:26915')
water2014_clip = gpd.clip(water2014_proj, metro_dissolve)


# Creating a list of the gpdf to loop through and find the smallest lake size
dfs = [water2014_clip, water2016_clip, water2018_clip]

# New field for impairment status in all data sets
for df in dfs:
    df["status"] = "Impaired"

def find_min(dfs):
    '''
    finds smallest lake within the impaired datasets
    Parameter: list of dataframes
    '''
    global minimum
    minimum = []
    for df in dfs:
        minimum.append(df["AREA_ACRES"].min())
    minimum = min(minimum)

# Calling function
find_min(dfs)

###
### CLEANING THE HYDROGRAPHY DATA SET
###

# Locate all invalid gometries and drop them from the dataset
hydro_drop_invalid = hydrography.loc[hydrography['geometry'].is_valid, :]

# Clipping hydro to the 7 county metro
hydro_clip = gpd.clip(hydro_drop_invalid, metro_dissolve)

# Narrowing down the number of features in the hydro layer to only lakes and ponds
hydro_lake = hydro_clip.loc[hydro_clip["wb_class"] == "Lake or Pond"]

# Selecting only the lakes that are at least the size of the the impaired water dataframes
hydro_lake = hydro_lake.loc[(hydro_lake["acres"] >= minimum)]

# Dropping all excess fields from the dataframe
hydro_clean = hydro_lake.drop(["fw_id", "dowlknum", "sub_flag", "wb_class", "lake_class", "shore_mi", "center_utm", "center_u_1",
                               "dnr_region", "fsh_office", "outside_mn", "delineated", "delineatio", "delineat_1", "delineat_2", 
                               "approved_b", "approval_d", "approval_n", "has_flag", "flag_type", "publish_da", "lksdb_basi", "has_wld_fl",
                               "wld_flag_t", "created_us", "created_da", "last_edite", "last_edi_1", "ow_use", "pwi_class", "map_displa", 
                               "shape_Leng", "shape_Area", "INSIDE_X", "INSIDE_Y", "in_lakefin"], axis = 1)

# New field for impairment status to be used when data is joined with the imparied data sets
hydro_clean["status"] = ""

###
### CLEANING THE 2020 IMPAIRED WATER DATA SET
###

# Load water 2020 data csv, selecting out the columns that we want and adding a geometry column
# and pulling out only the lake features.
water2020 = gpd.read_file("wq-iw1-65.csv")
water2020 = water2020[["Water body name", "AUID", "County", "Water body type", "geometry"]]
water2020_lake = water2020.loc[(water2020["Water body type"] == "Lake")]

# Dropping the "water body type" field since it is no longer needed
water2020_lake = water2020_lake[["AUID", "Water body name", "County", "geometry"]]

# Renaming the columns to match the two other datasets
water2020_lake = water2020_lake.rename(columns = {"Water body name" : "NAME", "County" : "COUNTY"})

# Selecting out the 7 county metro
counties = ["Anoka", "Hennepin", "Ramsey", "Washington", "Carver", "Scott", "Dakota"]
water2020_metro = water2020_lake.loc[(water2020_lake["COUNTY"].isin(counties))]

# Varifying all the correct counties are there
water2020_metro["COUNTY"].unique()

# Drop Duplicate AUIDs
water2020_clean = water2020_metro.drop_duplicates(subset = ["AUID"])



In [ ]:
unmatched2020 = water2020_clean.loc[water2020_clean['AUID'].isin(water2018_clip['AUID']) == False]
unmatched2020

In [ ]:
hydro_clean.head()

In [ ]:
names = unmatched2020.loc[unmatched2020["NAME"].isin(hydro_clean["pw_basin_n"]) == False]
names

In [ ]:
water2020_join_auid = water2020_clean.merge(water2018_clip, how = "left", on = "AUID")
water2020_join_auid

hydro_clean = hydro_clean.rename(columns = {"pw_basin_n" : "NAME_x"})

hydro_geometry = hydro_clean.loc[hydro_clean["NAME_x"] == "DeMontreville"]

hydro_geometry = hydro_geometry[["NAME_x", "geometry", "status"]]

water2020_join_name = water2020_join_auid.merge(hydro_geometry, how = "left", on = "NAME_x")

hydro_geometry2 = hydro_clean.loc[hydro_clean["NAME_x"] == "Laura"]

hydro_geometry2 = hydro_geometry2[["NAME_x", "geometry", "status"]]

water2020_join_name2 = water2020_join_name.merge(hydro_geometry2, how = "left", on = "NAME_x")
water2020_join_name2